UMLS METAMAP API FOR FITBIR DATA ELEMENTS


<br>
This script takes a FITBIR Data Element report, formats it for ingestion into MetaMap, and reformats MetaMap output for manual selection of relevant UMLS DE concepts and cuis. <br>
DESCRIPTION:<br>
    <br>
(1) Load FITBIR Data Element Report<br>
(2) Extract variable name and definition and remove stop words based on csv with list of stop words<br>
(3) Format FITBIR Data Elements for MetaMap ingestion<br>
(4) Initialize WebAPI settings<br>
(5) Call MetaMap WebAPI<br>
(6) Process MetaMap output for manual review and curation.<br>
(7) Check data elements for missing cuis, cuis not available, and multiple cuis<br>
BRICS CDES ATTRIBUTES USED TO FEED METAMAP:<br>
    <br>
- CDE title (minus stop words) - required<br>
- *CDE question text (minus stop words) â€“ optional; *only to use if the MetaMap output does not provide reasonable results; *currently not built into this script version 09/23/2022<br>
PRE-PROCESSING CDES BEFORE RUNNING THROUGH METAMAP:<br>
    <br>
The goal of this pre-processing is to make sure that MetaMap treats each row with CDE attributes as a separate piece of text. That includes:<br>
    <br>
1) Changing every cell delimiter to a semicolon or a period,<br>
2) Making sure there are no spurious line breaks, e.g.,<br>
3) Removing all punctuation marks<br>
4) Removing all stop words<br>
5) Adding an extra blank line between the text of each row<br>
6) Running the result as a plain text file<br>
META MAP SETTINGS:<br>
    <br>
1) knowledge source: 2020AB<br>
2) data version: USABase<br>
3) relaxed model: -C <br>
4) show candidates: -c<br>
5) show CUIs: -I<br>
6) show concept sources: -G<br>
7) formatted JSON output: --JSONf<br>
8) word-sense disambiguation: -y <br>
9) term processing: -z <br>
10) ignore word order: -i <br>
11) single-line-delimited input with ID: '--sldiID'<br>
    <br>
EXCEL WORKBOOK CONTENTS:<br>
    - sheet1: Candidate mappings for DEs --> df_maps<br>
    - sheet2: Original Data Element report --> df_DE_report<br>


LOAD PACKAGES AND DEFINE FUNCTIONS

In [2]:
import cui_mapping_functions as cmf # contains custom functions for loading/processing FITBIR csv datafiles
from skr_web_api import Submission
import os
import pandas as pd
import numpy as np
import json
import tkinter as tk  # for dialog boxes
from tkinter import filedialog
import importlib
root = tk.Tk()
root.withdraw()
importlib.reload(cmf)

ModuleNotFoundError: No module named 'skr_web_api'

MASTER SETTINGS

In [ ]:
rmv_stopwords_1_0 = 1
use_cheatsheet = 0
source_filter = None # not yet implemented, but eventually the script will allow you to query defined subsets of vocabularies

LOAD FITBIR DATA ELEMENT REPORT

In [3]:
fp_DE_report = filedialog.askopenfilename(
    title='Select dataElementExport.csv file')  # choose dataElementExport.csv file
df_DE_report = pd.read_csv(fp_DE_report)
dir_DE_report = os.path.dirname(fp_DE_report)
fn_DE_report_new = os.path.basename(dir_DE_report)
fs_nam = fn_DE_report_new.split('_')[0]

EXTRACT VARIABLE NAME AND TITLE AND REMOVE STOP WORDS

In [4]:
dir_step1 = cmf.create_folder(
    dir_DE_report + '/DE_Step-1_Format DEs and call MetaMap')
df_mm_DE_processing = df_DE_report[['variable name', 'title', 'definition', 'permissible values', 'permissible value descriptions',
                                    'preferred question text', 'domain.general (for all diseases)']].copy()

In [5]:
df_mm_DE_processing[['title_modified']] = df_mm_DE_processing['title'].apply(
    cmf.remove_punctuation)  # remove punctuation

In [6]:
if rmv_stopwords_1_0:
    # choose csv file with list of stopwords Olga curated
    fp_stopwords = filedialog.askopenfilename(
        title='Select MetaMap_Settings_StopWords.csv file')
    df_stopwords = pd.read_csv(fp_stopwords)
    # make lower case and remove leading/trailing whitespaces
    mm_stopwords = list(df_stopwords['Word'].str.lower().str.strip())
    # nltk_stopwords = stopwords.words('english')
    # nltk_stopwords.extend(mm_stopwords)
    df_mm_DE_processing[['title_modified', 'stopwords_removed']] = df_mm_DE_processing['title_modified'].apply(
        cmf.remove_stopwords, mm_stopwords=mm_stopwords)  # remove stopwords
else:
    df_mm_DE_processing['stopwords_removed'] = ''

FORMAT FITBIR DATA ELEMENTS FOR METAMAP INGESTION

In [7]:
df_mm_DE_processing['MetaMap_input'] = df_mm_DE_processing[[
    'variable name', 'title_modified']].agg('|'.join, axis=1)  # create MetaMap input
fp_df_mm_DE_processing = f"{dir_step1}/DE_step1_{fs_nam}_df_mm_DE_processing.xlsx"
# file you will have to manually review and specify whether to keep a row
writer = pd.ExcelWriter(fp_df_mm_DE_processing, engine='openpyxl')
sheet1 = 'MM_DE_PROCESSING'
df_mm_DE_processing.to_excel(writer, sheet_name=sheet1, index=False)
ws1 = writer.sheets[sheet1]
for i, width in enumerate(cmf.get_header_widths(df_mm_DE_processing)):
    excel_col_letter = cmf.get_column_letter(i+1)
    ws1.column_dimensions[excel_col_letter].width = width
writer.close()

In [8]:
df_mm_inputfile = df_mm_DE_processing['MetaMap_input']
fp_df_mm_inputfile = f"{dir_step1}/DE_step1_{fs_nam}_mm_inputfile_SingLinePMID.txt"
with open(fp_df_mm_inputfile, 'w') as f:
    for cell in df_mm_inputfile:
        f.write(cell)
        f.write("\n")

INITIALIZE API SETTINGS

In [9]:
email = 'armengolkm@nih.gov'
apikey = 'd507799b-ddc3-4592-b7b5-287fe6d3bf7c'
serverurl = "https://utslogin.nlm.nih.gov/cas/v1/tickets"
tgtserverurl = "https://utslogin.nlm.nih.gov/cas/v1/api-key"
serviceurl = 'https://ii.nlm.nih.gov/cgi-bin/II/UTS_Required/API_batchValidationII.pl'

In [10]:
cmd = 'metamap'
mm_data_year = '-Z 2020AB'
mm_data_version = '-V USAbase'
relaxed_model = '-C'
show_candidates = '-c'
n_mappings = '-f' # add
conjunctions = '--conj'
show_cuis = '-I'
ignore_word_order = '-i'
sources = '-G'
term_processing = '-z'
word_sense_disambiguation = '-y'
formatted_json_output = '--JSONf 2'
cmdargs = ' '.join([mm_data_year, mm_data_version, relaxed_model, show_candidates, show_cuis, ignore_word_order, sources,
                    term_processing,  word_sense_disambiguation, formatted_json_output])
inputfile = fp_df_mm_inputfile

CALL METAMAP WEP API

In [11]:
inst = Submission(email, apikey)
inst.set_serviceurl(serviceurl)
inst.init_generic_batch(cmd, cmf.unescape_string(cmdargs))
inst.set_batch_file(inputfile)
inst.form['SingLinePMID'] = 'yes'
inst.form['Batch_Command'] = '{} {}'.format(cmd, cmf.unescape_string(cmdargs))
response = inst.submit()
print('response status: {}'.format(response.status_code))
print('content: {}'.format(response.content.decode()))
if response.status_code == 200:
    jsondata0 = response.content.decode()
    print('content: {}'.format(jsondata0))
    # remove string "NOT DONE LOOP" if present
    jsondata = jsondata0.replace('NOT DONE LOOP', '')
    data = json.loads(jsondata)
    print('data:{}'.format(data))
    fp_json_output = f"{dir_step1}/DE_step1_{fs_nam}_mm_json_output.json"
    with open(fp_json_output, 'w') as chan:
        chan.write(jsondata)

response status: 200
content: NOT DONE LOOP
{ "AllDocuments":[
{
   "Document": {
     "CmdLine": {
       "Command": "metamap -L 2020AB -Z 2020AB --lexicon db -V USAbase -C -c -I -i -G -z -y --JSONf 2 -E /tmp/text_000N_39272 /tmp/text_000N.out_39272",
       "Options": [
         {
           "OptName": "lexicon_year",
           "OptValue": "2020AB"
         },
         {
           "OptName": "mm_data_year",
           "OptValue": "2020AB"
         },
         {
           "OptName": "lexicon",
           "OptValue": "db"
         },
         {
           "OptName": "mm_data_version",
           "OptValue": "USAbase"
         },
         {
           "OptName": "relaxed_model"
         },
         {
           "OptName": "show_candidates"
         },
         {
           "OptName": "show_cuis"
         },
         {
           "OptName": "ignore_word_order"
         },
         {
           "OptName": "sources"
         },
         {
           "OptName": "term_processing"
        

PROCESS METAMAP OUTPUT FOR MANUAL REVIEW AND CURATION

In [12]:
dict_docs = data["AllDocuments"]
ls_df_maps = []
cols_map = ['CandidateScore', 'CandidateCUI', 'CandidateMatched',
            'CandidatePreferred', 'MatchedWords', 'SemTypes', 'MatchMaps', 'IsHead',
            'IsOverMatch', 'Sources', 'ConceptPIs', 'Status', 'Negated']  # fields to extract for review
for doc in dict_docs:
    pmid = doc['Document']['Utterances'][0]['PMID']
    df_temp = pd.json_normalize(doc['Document']['Utterances'][0]['Phrases'], record_path=[
                                'Candidates'], errors='ignore')
    if df_temp.empty:
        df_temp = pd.DataFrame(columns=cols_map)
    df_temp.insert(0, 'PMID', pmid)
    ls_df_maps.append(df_temp)

In [13]:
df_maps = pd.concat(ls_df_maps) # combine separate dataframes for each PMID into one dataframe
df_maps['keep'] = np.nan
df_maps = pd.merge(df_DE_report[['variable name', 'title', 'definition', 'permissible value descriptions']], df_maps,
                   left_on='variable name', right_on='PMID', how='outer')  # adds data element metadata useful for review
df_maps[['CandidateCUI', 'CandidateMatched', 'CandidatePreferred', 'Sources']] = df_maps[[
    'CandidateCUI', 'CandidateMatched', 'CandidatePreferred', 'Sources']].fillna('Not Available')

In [14]:
fp_step1 = f"{dir_step1}/DE_step1_{fs_nam}_mm_UMLS_keepCol.xlsx"
writer = pd.ExcelWriter(fp_step1, engine='openpyxl')
sheet1 = 'DEs_UMLS_MetaMap'
sheet2 = 'DE_report'
df_maps.to_excel(writer, sheet_name=sheet1, index=False)
df_DE_report.to_excel(writer, sheet_name=sheet2, index=False)
ws1 = writer.sheets[sheet1]
for col in ['L', 'P']:
    # these columns are hidden as they aren't useful for review
    ws1.column_dimensions[col].hidden = True
writer.close()

MANUAL STEP - LABEL DE/UMLS CODES TO BE KEPT

In DE_Step-1_XYZ_mm_UMLS_keepCol.xlsx, you will need to manually add any character in the keep column to preserve that row for subsequent filtering.<br>
Keep saved as .xlsx file<br>
Do not change default sheet tab names